## Model Evaluation

In [1]:
import os

In [2]:
%pwd

'/home/utpal108/dev/Upwork/Projects/Diabetic-Retinopathy-Prediction/notebooks/diabetic_retinopathy'

In [3]:
os.chdir('../../')

In [4]:
%pwd

'/home/utpal108/dev/Upwork/Projects/Diabetic-Retinopathy-Prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path
import pandas as pd

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_path: Path
    training_images_path: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int
    evaluation_score_path: Path

In [6]:
from diabeticRetinopathy.constants import *
from diabeticRetinopathy.utils import create_directories, read_yaml, save_json

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        
        eval_config = EvaluationConfig(
            path_of_model =  Path(self.config.training.trained_model_path),
            training_path = Path(self.config.training.training_file_path),
            training_images_path = Path(self.config.training.training_images_path),
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE,
            evaluation_score_path = Path(self.config.evaluation.evaluation_score_path)
        )
        
        return eval_config

In [8]:
import tensorflow as tf

2024-04-16 13:31:02.625295: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-16 13:31:02.628776: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-16 13:31:02.672570: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-16 13:31:03.383473: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [9]:
class Evaluate:

    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        
        train_df = pd.read_csv(self.config.training_path)
        train_df['id_code'] = train_df['id_code'].apply(lambda x: x+'.png')
        train_df['diagnosis'] = train_df['diagnosis'].astype('str')

        train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255, 
            validation_split=0.2,
            horizontal_flip=True)
        

        self.valid_generator=train_datagen.flow_from_dataframe(
            dataframe=train_df,
            directory=self.config.training_images_path,
            x_col="id_code",
            y_col="diagnosis",
            batch_size=self.config.params_batch_size,
            class_mode="categorical", 
            target_size=self.config.params_image_size[:-1],
            subset='validation')

    def load_model(self, path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        score = model.evaluate(self.valid_generator)
        save_json(path = self.config.evaluation_score_path, data={'loss': score[0], 'accuracy': score[1]})


In [10]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluate(config=val_config)
    evaluation.evaluation()

except Exception as e:
    raise e

2024-04-16 13:31:14,186 : diabeticRetinopathy.logger - INFO - YAML file: config/config.yaml loaded successfully
2024-04-16 13:31:14,189 : diabeticRetinopathy.logger - INFO - YAML file: params.yaml loaded successfully
2024-04-16 13:31:14,190 : diabeticRetinopathy.logger - INFO - created directory at: artifacts
2024-04-16 13:31:15,431 : absl - WARNING - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.


Found 732 validated image filenames belonging to 5 classes.


/home/utpal108/dev/Upwork/Projects/Diabetic-Retinopathy-Prediction/venv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-04-16 13:31:20.107038: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 268435456 exceeds 10% of free system memory.
2024-04-16 13:31:20.347984: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 272646144 exceeds 10% of free system memory.
2024-04-16 13:31:20.517080: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 268435456 exceeds 10% of free system memory.
2024-04-16 13:31:20.812518: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 268435456 exceeds 10

46/46 ━━━━━━━━━━━━━━━━━━━━ 219s 5s/step - accuracy: 0.7886 - loss: 0.2381


2024-04-16 13:34:56,995 : diabeticRetinopathy.logger - INFO - json file saved at: artifacts/training/scores.json
